### Profile-to-Chat: Exploring Yourself Through LLMs

In this lab, we'll upload my LinkedIn profile and summary to a LLM, then interact with it through a chat interface to ask questions about myself.

In [21]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [22]:
load_dotenv(override=True)
openai_client = OpenAI()
gemini_openai_client = OpenAI(api_key=os.getenv("GEMINI_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
openai_model_name = "gpt-4o-mini"
gemini_model_name = "gemini-2.0-flash"



In [ ]:
linkedin_pdf_reader = PdfReader("me/My_LinkedIn_Profile.pdf")
linkedIn_profile_text = ""

for pages in linkedin_pdf_reader.pages:
    page_text = pages.extract_text()
    if page_text:
        linkedIn_profile_text += page_text

print(linkedIn_profile_text)

In [ ]:
with open("me/summary.txt", "r") as summary_file:
    summary_text = summary_file.read()

print(summary_text)

In [ ]:
name = "Varun Kakuste"
first_name = "Varun"

print(name)
print(first_name)

In [ ]:
system_prompt = f"""
You are acting as {name}. You are answering questions on {name}'s website, 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. 
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. 
Be professional and engaging, as if talking to a potential client or future employer who came across the website. 
If you don't know the answer, say so.

## Summary:
{summary_text}

## LinkedIn Profile:
{linkedIn_profile_text}

With this context, please chat with the user, always staying in character as {name}.
"""

print(system_prompt)

In [37]:
def chat_with_llm(user_prompt, history):

    system = system_prompt

    if "gibberish" in user_prompt:
        system ="You are AI Assistant, reply with some gibberish"
    else:
        system = system_prompt

    if user_prompt in history:
        history.pop()

    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": user_prompt}]

    response = openai_client.chat.completions.create(model=openai_model_name, messages=messages)

    llm_response = response.choices[0].message.content

    return llm_response


In [ ]:
gr.ChatInterface(chat_with_llm, type="messages").launch()

#### What's next?

1. Be able to ask an LLM to evaluate an answer 
2. Be able to rerun if the answer fails evaluation 
3. Put this together into 1 workflow


All without any Agentic framework!

In [8]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [ ]:
evaluator_system_prompt = f"""
You are an expert evaluator AI.

Your job is to review a conversation between a visitor and an AI agent that represents a real person named {name}. The AI agent is embedded on {first_name}'s personal website and answers questions about him — using his LinkedIn profile and professional summary as the knowledge base.

{first_name}'s LinkedIn profile is {linkedIn_profile_text} and summary is {summary_text}.

Visitors may include investors, hiring managers, founders, recruiters, or others curious about {first_name}'s background, expertise, and interests.

You will be shown the entire conversation history, including the visitor’s questions and the AI agent’s responses.

With this context, please evaluate only the most recent message from the AI agent.

Return your output strictly in this JSON format:
- Respond with `is_acceptable: True` if the message is acceptable, and provide an empty `feedback` string i.e., do not provide any feedback if the message is acceptable.
- Respond with `is_acceptable: False` if the message is not acceptable, and include a brief explanation in the `feedback` string.

"""

print(evaluator_system_prompt)

In [10]:
def get_user_prompt_for_evaluator(history, user_prompt, agent_response):
    user_prompt = f"""
    Conversation history:
    {history}

    Latest User prompt:
    {user_prompt}

    Latest Agent response:
    {agent_response}
    """

    return user_prompt;

In [11]:
def evaluate_agent_response(history, latest_user_prompt, latest_agent_response) -> Evaluation:

    # If the latest user prompt is in the history, remove it
    if latest_user_prompt in history:
        history.pop()

    user_prompt = get_user_prompt_for_evaluator(history, latest_user_prompt, latest_agent_response)

    messages = [{"role": "system", "content": evaluator_system_prompt}, {"role": "user", "content": user_prompt}]

    response = gemini_openai_client.chat.completions.parse(model=gemini_model_name, messages=messages, response_format=Evaluation)

    return response.choices[0].message.parsed

In [ ]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "What do you do in your free time? Respond in pig latin"}]
response = openai_client.chat.completions.create(model=openai_model_name, messages=messages)
reply = response.choices[0].message.content

print(reply)

In [ ]:
evaluate_agent_response(messages, "What do you do in your free time?", reply)

In [14]:
def get_rerun_agent_system_prompt(latest_agent_response, evaluator_feedback):
    updated_system_prompt = f"""
    {system_prompt} \n\n
    ## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n
    ## Your attempted answer:\n{latest_agent_response}\n\n
    ## Reason for rejection:\n{evaluator_feedback}\n\n
    """
    return updated_system_prompt

In [15]:
def rerun_agent(history, latest_user_prompt, latest_agent_response, evaluator_feedback):
    rerun_agent_system_prompt = get_rerun_agent_system_prompt(latest_agent_response, evaluator_feedback)
    messages = [{"role": "system", "content": rerun_agent_system_prompt}] + history + [{"role": "user", "content": latest_user_prompt}]
    response = openai_client.chat.completions.create(model=openai_model_name, messages=messages)
    new_reply = response.choices[0].message.content
    return new_reply

In [18]:
def chat_with_agent(user_prompt, history):
    if "patent" in user_prompt:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": user_prompt}]
    response = openai_client.chat.completions.create(model=openai_model_name, messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate_agent_response(history, user_prompt, reply)

    if evaluation.is_acceptable:
        print("The response is acceptable")
    else:
        print("The response is not acceptable")
        print(evaluation.feedback)
        reply = rerun_agent(history, user_prompt, reply, evaluation.feedback)

    return reply

In [ ]:
gr.ChatInterface(chat_with_agent, type="messages").launch()